# Table of Contents
* [Load data](#Load-data)
* [dataset validation](#dataset-validation)
* [dataset stats](#dataset-stats)
	* [topic names](#topic-names)
	* [question type dist](#question-type-dist)
	* [looking for missing values](#looking-for-missing-values)
* [html rendering](#html-rendering)
* [building word corpus](#building-word-corpus)
* [End](#End)


In [102]:
%%capture
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, OrderedDict

import cv2
import pprint
import pickle
import json
import requests
import io
import sys
import os
from binascii import b2a_hex
import base64
from wand.image import Image as WImage
from IPython.display import display
import PIL.Image as Image
from copy import deepcopy
import glob

import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import jsonschema
# from pdfextraction.ck12_new_schema import ck12_schema


In [103]:
%%capture
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pylab as plt
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light solarized
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['figure.figsize'] = (16.0, 10.0)

# Load data

In [85]:
dataset_root_dir = '/Users/schwenk/wrk/stb/dataset_releases/data_release_beta_5_p5_11_7/'
data_file = 'tqa_dataset_beta5p5.json'

with open(os.path.join(dataset_root_dir, data_file), 'r') as f:
    ck12_combined_dataset_raw = json.load(f)

In [86]:
ck12_combined_dataset = deepcopy(ck12_combined_dataset_raw)

# dataset validation

In [303]:
from pdfextraction.validate_and_split import DataSetIntegrityChecker

In [184]:
dataset_validator = DataSetIntegrityChecker(dataset_root_dir, data_file)
terr = dataset_validator.validate_dataset() 
if terr == 'all validation test passed':
    print(terr)

In [185]:
img_path_errors = [err for err in terr['image_paths'] if 'textbook' not in err]

In [186]:
len(img_path_errors)

504

# test train splits

In [363]:
from pdfextraction.validate_and_split import TestTrainSplitter
test_train_splitter = TestTrainSplitter(dataset_root_dir, data_file)

In [364]:
manual_assignments = {"ML_0289": {"lesson_name": "flow of energy", "split": "test"}, "ML_0505": {"lesson_name": "insect reproduction and life cycle", "split": "train"}}
# tt_split, debug_tt_splits = test_train_splitter.perform_split(manual_assignments=manual_assignments)

In [366]:
%%time
split_trials = [test_train_splitter.split_and_compute_stats(manual_assignments) for i in range(5)]    
split_trials_test_fracts = [{k:v['test_fraction'] for k, v in trial.items()} for trial in split_trials]

CPU times: user 10.1 s, sys: 10.5 ms, total: 10.1 s
Wall time: 10.2 s


In [367]:
split_trial_df = pd.DataFrame(split_trials_test_fracts)
split_trial_df = split_trial_df.apply(pd.to_numeric)
split_trial_df.describe()

,n_diagram_questions,n_instructional_diagrams,n_text_questions,n_topics
count,5.000000,5.000000,5.000000,5.000000
mean,0.270200,0.229600,0.201600,0.192200
std,0.084851,0.042835,0.018243,0.012657
min,0.197000,0.163000,0.173000,0.172000
25%,0.225000,0.217000,0.199000,0.190000
50%,0.231000,0.239000,0.203000,0.196000
75%,0.288000,0.254000,0.211000,0.197000
max,0.410000,0.275000,0.222000,0.206000


## hide

In [319]:
len(tt_split['test'] + tt_split['train'])

1076

In [315]:
print('L_0575' in tt_split['test'])
print('L_0575' in tt_split['train'])

test_st = 'te'
test_st.format()


manual_assignments.keys()

test_train_splitter.compute_split_stats(tt_split)

False
True


In [261]:
test_st = 'te'
test_st.format()

manual_assignments = {"ML_0289": {"lesson_name": "flow of energy", "split": "test"}, "ML_0505": {"lesson_name": "insect reproduction and life cycle", "split": "train"}}

manual_assignments.keys()

'te'

In [8]:
def record_validation_errors(dataset, schema):
    qs_removed = []
    validator = jsonschema.Draft4Validator(schema)
    for error in sorted(validator.iter_errors(dataset), key=lambda x: x.absolute_path[0]):
        print(error)
        lesson, quest, question_class, q_number = list(error.absolute_schema_path)[:4]
        problem_q_section = dataset[subject][lesson][quest][question_class]
        if q_number in problem_q_section.keys():
            print(dataset[subject][lesson][quest][question_class].pop(q_number))
            qs_removed.append(dataset[subject][lesson][quest][question_class].pop(q_number))
    return qs_removed

In [167]:
# ml_ids_train, ml_ids_test, debug_train = test_train_splitter.perform_split()

# ml_ids_train, ml_ids_test, debug_train = test_train_splitter.perform_split()

In [9]:
validator = jsonschema.Draft4Validator(new_schema)
for error in sorted(validator.iter_errors(ck12_combined_dataset_raw), key=lambda x: x.absolute_path[0]):
    print(error.message)
    print(error.absolute_path)
    print()

NameError: name 'new_schema' is not defined

# dataset stats

## topic names

In [ ]:
es_lesson_names = [item for sublist in [val['topics'].keys() for val in ck12_combined_dataset['earth-science'].values()] for item in sublist]
ps_lesson_names = [item for sublist in [val['topics'].keys() for val in ck12_combined_dataset['physical-science'].values()] for item in sublist]
ls_lesson_names = [item for sublist in [val['topics'].keys() for val in ck12_combined_dataset['life-science'].values()] for item in sublist]

combined_topics = es_lesson_names + ps_lesson_names + ls_lesson_names
topic_series = pd.Series(combined_topics).value_counts()
# topic_series[:18]
# topic_series[18:40]

In [ ]:
topic_series[:20]

In [ ]:
len(combined_topics)

In [ ]:
len(topic_series)

__earth science topics__

In [ ]:
sorted(ck12_combined_dataset['earth-science'].keys())

__life science topics__

In [ ]:
sorted(ck12_combined_dataset['life-science'].keys())

__physical science topics__

In [ ]:
sorted(ck12_combined_dataset['physical-science'].keys())

In [ ]:
 all_lessons = ck12_combined_dataset['physical-science'].keys() + ck12_combined_dataset['life-science'].keys() + ck12_combined_dataset['physical-science'].keys() +ck12_combined_dataset['earth-science'].keys()
print len(all_lessons)

In [ ]:
ck12_combined_dataset.values()

In [ ]:
topic_list = [subject[lesson]['topics'].keys() for subject in ck12_combined_dataset.values() for lesson in subject]
flattened_topics = [topic for lesson in topic_list for topic in lesson]
len(flattened_topics)

In [ ]:
len(set(flattened_topics))

## question type dist

prior to quizes

In [ ]:
# q_types = []
# for subject, flexbook in ck12_combined_dataset.items():
#     for lesson in flexbook.values():
#         for question in lesson['questions']['nonDiagramQuestions'].values():
#             q_types.append(question['type'])
# question_counts = pd.Series(q_types).value_counts()
# print 'total number of questions = ' + str(question_counts.sum())
# question_counts

post quizes

In [ ]:
q_types = []
for subject, flexbook in ck12_combined_dataset.items():
    for lesson in flexbook.values():
        for question in lesson['questions']['nonDiagramQuestions'].values():
            q_types.append(question['type'])
question_counts = pd.Series(q_types).value_counts()
print 'total number of questions = ' + str(question_counts.sum())
question_counts

In [ ]:
_ = question_counts.plot(kind="barh")
plt.title('Question Format Distribution', fontsize=50, verticalalignment='bottom', color = b16_colors.b)
plt.ylabel("Question type", fontsize=30, labelpad=10, color = b16_colors.b)
plt.xlabel("Number of unique questions", fontsize=30, labelpad=10, color = b16_colors.b)
plt.tick_params(axis='x', which='major', labelsize=20)
plt.tick_params(axis='y', which='major', labelsize=20)

In [ ]:
usable_questions = question_counts[:2].append(question_counts[3:4])

In [ ]:
print 'questions usable immedeatly = ' + str(usable_questions.sum())
usable_questions

In [ ]:
for subject, flexbook in ck12_combined_dataset.items():
    q_types = []
    for lesson in flexbook.values():
        for question in lesson['questions']['nonDiagramQuestions'].values():
            q_types.append(question['type'])
    question_counts = pd.Series(q_types).value_counts()
    print 'total number of ' + subject + ' questions = ' + str(question_counts.sum())
    print question_counts
    print 

## looking for missing values

In [ ]:
for subject, flexbook in flexbook_ds.items():
    q_len = []
    for lesson_name, lesson in flexbook.items():
        q_len.append(len(lesson['questions']['nonDiagramQuestions'].values()))
        if q_len[-1] == 7:
            print  subject, lesson_name
#             pprint.pprint(lesson['questions']['nonDiagramQuestions'])
    q_lengths = pd.Series(q_len).value_counts()
    print 'total number of ' + subject + ' lessons = ' + str(q_lengths.sum())
    print q_lengths
    print 

The lessons with fewer questions seem to be genuine, i.e. those are the number of questions in the workbook

# html rendering

### old nested format

In [ ]:
from IPython.core.display import HTML

In [ ]:
for subject, flexbook in ck12_combined_dataset.items():
    for lesson_name, lesson in flexbook.items():
        if lesson['questions']['diagramQuestions']:
            lesson_html = display_lesson_html(ck12_combined_dataset[subject], lesson_name, 'questions')
            with open(os.path.join('review_diagram_questions', lesson_name + '.html'), 'w') as f:
                f.write(lesson_html.encode('ascii', 'ignore').decode('utf-8'))

In [ ]:
for subject, flexbook in ck12_combined_dataset.items():
    for lesson_name, lesson in flexbook.items():
        if 'instructionalDiagrams' in lesson.keys():
            lesson_html = display_lesson_html(ck12_combined_dataset[subject], lesson_name, 'descriptions')
            with open(os.path.join('review_diagram_descriptions', lesson_name + '.html'), 'w') as f:
                f.write(lesson_html.encode('ascii', 'ignore').decode('utf-8'))

### new flat format

In [ ]:
# display_lesson_html(ck12_combined_dataset[20], ck12_combined_dataset[20]['lessonName'], 'lessons', out_path)

In [ ]:
%%writefile render_html.py
def render_html_from_dataset(path_to_data_json):
    
    with open(output_dir + 'build_v5f.json', 'r') as f:
        ck12_combined_dataset = json.load(f)
    
    out_path = '../html_renders' 
    render_types = ['lessons', 'questions', 'descriptions']
    for render in render_types:
        for lesson in ck12_combined_dataset:
            if render == 'lessons':
                lesson_html = display_lesson_html(lesson, lesson['lessonName'], render, out_path)
                with open(os.path.join(out_path, render, lesson['lessonName'].replace(' ', '_') + '.html'), 'w') as f:
                    f.write(lesson_html.encode('ascii', 'ignore').decode('utf-8'))
            elif lesson['questions']['diagramQuestions']:
                lesson_html = display_lesson_html(lesson, lesson['lessonName'], render, out_path)
                with open(os.path.join(out_path, render, lesson['lessonName'].replace(' ', '_') + '.html'), 'w') as f:
                    f.write(lesson_html.encode('ascii', 'ignore').decode('utf-8'))

In [ ]:
render_html_from_dataset('./output_data_from_nbs/build_v5f.json')

# building science word corpus

In [ ]:
flexbook_ds['life-science'].values()[0]['topics'].values()[0]

In [ ]:
vocab_sections = ['Lesson Vocabulary', 'Vocabulary']
flexbook_vocab = set()

for subject in flexbook_ds.values():
    for lesson in subject.values():
        for topic_name, topic in lesson['topics'].items():
            if topic_name in vocab_sections:
                vocab_section = topic['content']['text']
                if '\n' in vocab_section:
                    flexbook_vocab.update(vocab_section.lower().split('\n'))
                elif ' ' in vocab_section:
                    flexbook_vocab.update(vocab_section.lower().split())
                else:
                    print 'no vocab'

In [ ]:
len(flexbook_vocab)

In [ ]:
from nltk.tokenize import wordpunct_tokenize
cached_sw = stopwords.words("english") + list(string.punctuation)

In [ ]:
vocab_list = list(flexbook_vocab)

In [ ]:
seperated_list = [word.replace('(', ' ').replace(')', ' ').split() for word in vocab_list]

In [ ]:
flattened_list = [item for sublist in seperated_list for item in sublist if len(item) > 3 and item not in cached_sw]

In [ ]:
len(flattened_list)

In [ ]:
with open('ck_12_vocab_words.pkl', 'w') as f:
    pickle.dump(flattened_list, f)

Use entire corpus

In [ ]:
flexbook_vocab = ''

for subject in flexbook_ds.values():
    for lesson in subject.values():
        for topic_name, topic in lesson['topics'].items():
            vocab_section = topic['content']['text']
            flexbook_vocab += ' ' + vocab_section.lower()

In [ ]:
len(flexbook_vocab)

In [ ]:
fb_tokens = wordpunct_tokenize(flexbook_vocab)
normalized_tokens = [toke.strip().lower().encode('ascii', 'ignore').decode() for toke in fb_tokens if toke not in cached_sw]

In [ ]:
tb_freq_d = nltk.FreqDist(normalized_tokens)
most_common_fb_words = tb_freq_d.most_common()

In [ ]:
fb_entire_corpus_vb = set(normalized_tokens)

In [ ]:
with open('ck_12_all_words.pkl', 'w') as f:
    pickle.dump(fb_entire_corpus_vb, f)

In [ ]:
len(fb_entire_corpus_vb)

# End

In [ ]:
import jinja2
from IPython.core.display import HTML

jnjenv = jinja2.Environment()

%%writefile lesson_viz.py
def make_lesson_data(lesson_json):
    nested_text = []    
    for topic, content in sorted(lesson_json['topics'].items(), key=lambda (k,v): v['orderID']):
        nested_text.append((topic, content['content']['text']))
    return nested_text

def make_page_html(lesson_data, page_html):
    return jnjenv.from_string(page_html).render(lesson=lesson_data[0], topics=lesson_data[1])

def display_lesson_html(flexbook, lesson):
    lesson_json = flexbook[lesson]
    lesson_data = (lesson, make_lesson_data(lesson_json))
    lesson_html = make_page_html(lesson_data, page_html)
    return HTML(lesson_html)

page_html = """
<!DOCTYPE html>
<html>
  <head>
    <style type="text/css">
    </style>
  </head>
  <body>
    <div class="container">
      <h1>Lesson: {{lesson}}</h1>
      <ul>
        {% for topic in topics %}
        <p>
        </p>
        <h3>{{topic.0}}</h3>
        <p>{{
        topic.1
        }}</p>
        {% endfor %}
      </ul>
    </div>
    <script src="http://code.jquery.com/jquery-1.10.2.min.js"></script>
    <script src="http://netdna.bootstrapcdn.com/bootstrap/3.0.0/js/bootstrap.min.js"></script>
  </body>
</html>
"""

In [ ]:

stat_data = {'Number of Entities':stats_counter, 'Average Number per image': stats_fract}
count = 2
html = "<table>"
# add header row

html += "<tr><th>"
for k in stat_data.keys():
    html += "<th>"+k

html += "<tr><th>Entity Category"
for j in range(count):
    html += "<th>"

for k, v in stats_counter.items():
    html += "<tr><th>"+k
#     for j in range(count):
    html += "<td>" + str(v)
    html += "<td>" + "%.2f" % stats_fract[k]
html += '<tr>'
    
# for k, v in stats_fract.items():
#     html += "<tr><th>"+k
#     for j in range(count):
#         html += "<td>" + str(v)
        
html += "</table>"
HTML(html)

In [ ]:
page_html = """
<html>
<head>
<title>{{ title }}</title>
</head>
<body>
Hello.
</body>
</html>
"""

In [ ]:
page_template = """
<html>
 <head>
  <title>KB HIT</title>
  <meta content='text/html'/>
  <script type='text/javascript' src='https://s3.amazonaws.com/mturk-public/externalHIT_v1.js'></script>
 </head>
 <body>
    <p>We are constructing a large knowledge base (KB) about elementary science and commonsense knowledge, to help computers answer questions more reliably. We are planning to release the KB as a free, open source resource for the community when it is complete. Your work here will help us assemble this KB and contribute to this effort.</p>

    <p>Below, the computer has automatically extracted some candidate facts from text for possible inclusion in the KB. However, some are weird, false, or nonsensical. This task will help us distinguish the good facts, to include in the KB, from the bad.</p>
     <form name='mturk_form' method='post' id='mturk_form' action='https://workersandbox.mturk.com/mturk/externalSubmit'>
      <input type='hidden' value='' name='assignmentId' id='assignmentId'/>		 
      <table>
        <tr><th></th><th>Commonsense Knowledge</th></tr>
        {% for n in input_data %}
            <tr><td>{{n.sentence}}</td><td nowrap>
            <!--these break-->
            <!--<input type="hidden" name="{{n.sentence_id}}" id="assignmentId" value="ASSIGNMENT_NOT_AVAILABLE" />-->
            <!--<input type="hidden" name="assignmentId" id="assignmentId" value="ASSIGNMENT_NOT_AVAILABLE" />-->
            <!--this is in the official documentation but breaks anyway!-->
            <!--<input type='hidden' value='' name='assignmentId' id='assignmentId'/>-->
            <!--this works:-->
            <input name="{{n.sentence_id}}" type="radio" value="true-act" />EXPECTED ACTION
            <input name="{{n.sentence_id}}" type="radio" value="false-act" />RARE/FALSE ACTION
            <input name="{{n.sentence_id}}" type="radio" value="true-prop" />TRUE PROPERTY
            <input name="{{n.sentence_id}}" type="radio" value="false-prop" />RARE/FALSE PROPERTY
            <input name="{{n.sentence_id}}" type="radio" value="nonsense" />NONSENSE
            <input name="{{n.sentence_id}}" type="radio" value="unknown" />DON'T KNOW
            </td></tr>
        {% endfor %}
      </table>
      <p><input type="submit" id="submitButton" value="Submit" /></p>
   </form>
  <script language="Javascript">turkSetAssignmentID();</script>
 </body>
</html>

"""

In [ ]:
subject = 'life-science' 
# lesson = '15.1 Understanding Animal Behavior'
random_lesson = np.random.choice(flexbook_ds[subject].keys(), 1)[0]
display_lesson_html(flexbook_ds[subject], random_lesson)